<a href="https://colab.research.google.com/github/prisnormando/basicpythonworkshop/blob/main/digitalObjectsAlgorithmCurator_DirFilesInspectionAndReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Referências

### Links
https://docs.python.org/pt-br/3.11/whatsnew/3.11.html#operator
https://www.w3schools.com/python/default.asp
https://www.gov.br/cidadania/pt-br/servicos/sagi/microdados
https://docs.dataprep.ai/index.html
https://towardsdatascience.com/exploratory-data-analysis-in-python-a-step-by-step-process-d0dfa6bf94ee


### Estrutura storage AESOP HPC
* raw
* refined
* shared
* transient
* trusted

nas pastas domínios de conhecimento

### domínios de conhecimento
* bioclimatico
* clínica
* metagenômica
* midiasocial
* modelagem
* otc
* socioeconomico





---



# Inspeção das pastas do storage

Importa os módulos para listagem e manipulação dos arquivos.

In [ ]:
import os
import time
import pandas as pd
import dataprep

Monta o diretório de arquivos (neste caso o Google Drive).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Verifica o diretório atual

In [ ]:
print(os.getcwd())

Cria uma lista com os diretório e arquivos de acordo com o caminho indicado na variável 'start_path'.

In [ ]:
def list_files(startpath):
    report = "<html>\n<head>\n<title>Relatório de diretórios e arquivos</title>\n</head>\n<body>\n<h1>Relatório de diretórios e arquivos</h1>\n<ul>"
    for root, dirs, files in os.walk(startpath):
        report += "<li>" + os.path.basename(root) + "/\n<ul>"
        for d in dirs:
            report += "<li>" + d + "/</li>"
        for f in files:
            report += "<li>" + f + "</li>"
        report += "</ul></li>"
    report += "</ul>\n</body>\n</html>"
    return report

start_path = '/content/drive/MyDrive/Empresa/Doutora/eventos'
report = list_files(start_path)

with open("report.html", "w") as f:
    f.write(report)


Cria um relatório HTML com os diretórios e arquivos de acordo com o caminho indicado na variável 'start_path'.

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

start_path = '/'
lista = list_files(start_path)
print(lista)

#Metadados dos objetos digitais

Este script seleciona as pastas e arquivos em um caminho indicado, extrai os metadados dos arquivos e, em seguida, cria uma relatório HTML.

In [ ]:
def get_file_info(file_path):
    stat = os.stat(file_path)
    creation_time = time.ctime(stat.st_ctime)
    modification_time = time.ctime(stat.st_mtime)
    size = stat.st_size
    return (creation_time, modification_time, size)

def list_files(startpath):
    report = "<html>\n<head>\n<title>Relatório de diretórios e arquivos</title>\n<style>\n"
    report += "ul { list-style-type: none; }\n"
    report += "li span {\n  display: inline-block;\n  width: 200px;\n}\n"
    report += "li {\n  margin-bottom: 10px;\n}\n"
    report += "</style>\n</head>\n<body>\n<h1>Relatório de diretórios e arquivos</h1>\n<ul>"
    for root, dirs, files in os.walk(startpath):
        report += "<li><span>" + os.path.basename(root) + "/</span>\n"
        report += "<ul>"
        for d in dirs:
            dir_path = os.path.join(root, d)
            creation_time, modification_time, size = get_file_info(dir_path)
            report += "<li><span>" + d + "/</span>\n"
            report += "<ul><li><span>Criado em: </span>" + creation_time + "</li>"
            report += "<li><span>Modificado em: </span>" + modification_time + "</li>"
            report += "<li><span>Tamanho da pasta: </span>" + str(size) + " bytes</li></ul></li>"
        for f in files:
            file_path = os.path.join(root, f)
            creation_time, modification_time, size = get_file_info(file_path)
            ext = os.path.splitext(f)[1]
            report += "<li><span>" + f + "</span>\n"
            report += "<ul><li><span>Criado em: </span>" + creation_time + "</li>"
            report += "<li><span>Modificado em: </span>" + modification_time + "</li>"
            report += "<li><span>Tamanho do arquivo: </span>" + str(size) + " bytes</li>"
            report += "<li><span>Extensão: </span>" + ext + "</li></ul></li>"
        report += "</ul></li>"
    report += "</ul>\n</body>\n</html>"
    return report

start_path = '/sample_data'
report = list_files(start_path)
with open("report.html", "w") as f:
    f.write(report)


#Relatório a partir da listagem realizada via Bash

autor: Eric Maia - Engenheiro de dados (Membro da curadoria do Cidacs)

O script toma a listagem padrão criada a partir do comando 'ls' sobre um caminho de pastas (árvore) no Windows.

Em seguida, os dados tratados são exportados para Excel.

In [ ]:
# Caminho do arquivo no drive
caminho = '/content/drive/MyDrive/Robespierre/Curadoria/'
arquivo = 'MapeamentoParaTeste.csv'


# Codificação da leitura padrão é utf-8, 
# mas observou-se, no notepad, que a codificação deste é utf-16
f = open(caminho+arquivo,encoding='utf-16') # Lê o arquivo em utf-16

# Quebra o arquivo em uma lista de linhas
linhas = f.readlines()

In [ ]:
# Cria lista
lista = []

for linha in linhas: # Lê cada linha
  if linha.isspace():
    try:
      if dict_pasta: # testa se já existe um dicionário com as informações da última pasta lida
        lista.append(dict_pasta) # adiciona à lista      
        del dict_pasta # exclui
    except:
      continue

  if linha.startswith('    Diretório: '): # verifica se contem caminho de diretório
    diretorio = linha.replace('\n','').split('Diretório: ')[-1] # remove quebra de linha e extrai apenas o caminho
    continue

  if linha.startswith('Mode'): # verifica se é cabeçalho do conteúdo da pasta
    cabecalho = linha.split() # extrai cada rótulo do cabeçalho que será chave do conteúdo no dicionário

    dict_pasta = {} # cria um novo dicionário pra armazenar informações da pasta
    dict_pasta['diretorio'] = diretorio # cria chave diretorio no dicionario e salva o caminho do diretório
    continue

  if (linha.startswith('-a-') | linha.startswith('d--')): # verifica se é linha de informação do conteúdo
    conteudo = linha.split()
    
    # verifica se linha quebrou em espaços no nome do arquivo e rejunta o nome
    if len(conteudo) > 4:
      conteudo = [conteudo[0],' '.join(conteudo[1:3]),conteudo[3],' '.join(conteudo[4:])]

    # pra cada item da linha, cria uma chave com o nome encontrado no cabeçalho e preenche com o conteúdo
    for i,item in enumerate(conteudo): 
      dict_pasta[cabecalho[i]] = item
    # dict_pasta['conteudo'].append(dict_conteudo) # adiciona na lista de conteúdo da pasta
    continue

  else: 
    continue

In [ ]:
len(lista)

61

In [ ]:
df = pd.DataFrame(lista)
df.to_csv(caminho+'MapeamentoTratado.csv',index=False)
df.to_excel(caminho+'MapeamentoTratado.xlsx',index=False)
df

,diretorio,Mode,LastWriteTime,Length,Name
0,C:\Users\luiz.farias\Desktop,-a----,26/05/2022 13:33,2483,Word.lnk
1,C:\Users\luiz.farias\Desktop\Apresentações,-a----,08/08/2022 10:28,872269,TED 159 Perguntas Proposta Seminário Interno....
2,C:\Users\luiz.farias\Desktop\backup luiz 12052022,NaN,NaN,NaN,NaN
3,C:\Users\luiz.farias\Desktop\backup luiz 12052...,-a----,16/12/2021 09:27,64559,[Entrada]_Arquivos_NPD_-_16_12_2021.zip
4,C:\Users\luiz.farias\Desktop\backup luiz 12052...,-a----,21/02/2022 10:40,284532,5.jpeg
...,...,...,...,...,...
56,C:\Users\luiz.farias\Desktop\teste\themes,-a----,20/06/2022 10:31,175205,Zenburn.xml
57,C:\Users\luiz.farias\Desktop\teste\updater,-a----,27/03/2022 16:05,133872,updater.ico
58,C:\Users\luiz.farias\Desktop\teste\userDefineL...,-a----,05/10/2021 19:39,6439,markdown._preinstalled_DM.udl.xml
59,C:\Users\luiz.farias\Desktop\Textos,-a----,21/07/2022 10:45,376427,Textualização apresentação setor de Curadoria....


# Análise Exploratória dos Dados nos Arquivos

### Passos

1. Importing a dataset
2. Understanding the big picture
3. Preparation
4. Understanding of variables
5. Study of the relationships between variables
6. Brainstorming
7. Report
8. Clean
9. Harmonize
10. Modeling

Carregar o arquivo na pasta

In [ ]:
#Load the required libraries
# data manipulation
import pandas as pd
import numpy as np

# data viz
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

# apply some cool styling
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12,  6)

# use sklearn to import a dataset
from sklearn.datasets import load_wine

In [ ]:
#variavel
wine = load_wine()

#Load the data
df = pd.read_csv('titanic.csv')

#View the data
df.head()

Informações básicas sobre a base

In [ ]:
#Basic information

df.info()

#Describe the data

df.describe()

Valores duplicados

In [ ]:
#Find the duplicates

df.duplicated().sum()
.isduplicated().sum()


Valores únicos

In [ ]:
#unique values

df['Pclass'].unique()

df['Survived'].unique()

df['Sex'].unique()

In [ ]:
#Plot the unique values

sns.countplot(df['Pclass']).unique()

Valores nulos

In [ ]:
#Find null values

df.isnull().sum()

Substituição dos nulos

In [ ]:
#Replace null values

df.replace(np.nan,'0',inplace = True)

#Check the changes now
df.isnull().sum()

Tipos de dados

In [ ]:
#Datatypes

df.dtypes

Filtrar os dados

In [ ]:
#Filter data

df[df['Pclass']==1].head()

Boxplot

In [ ]:
#Boxplot

df[['Fare']].boxplot()

df.target.value_counts().plot(kind="bar")
plt.title("Value counts of the target variable")
plt.xlabel("Wine type")
plt.xticks(rotation=0)
plt.ylabel("Count")
plt.show()

In [ ]:
print(f"Skewness: {df['magnesium'].skew()}")
print(f"Kurtosis: {df['magnesium'].kurt()}")

Correlação

In [ ]:
#Correlation 

df.corr()

In [ ]:
#Correlation plot

sns.heatmap(df.corr())

sns.catplot(x="target", y="proline", data=df, kind="box", aspect=1.5)
plt.title("Boxplot for target vs proline")
plt.show()

sns.scatterplot(x="proline", y="flavanoids", hue="target", data=df, palette="Dark2", s=80)
plt.title("Relationship between proline, flavanoids and target")
plt.show()

corrmat = df.corr()
hm = sns.heatmap(corrmat, 
                 cbar=True, 
                 annot=True, 
                 square=True, 
                 fmt='.2f', 
                 annot_kws={'size': 10}, 
                 yticklabels=df.columns, 
                 xticklabels=df.columns, 
                 cmap="Spectral_r")
plt.show()

# Limpeza

Renomear variáveis

df.rename(columns={"od280/od315_of_diluted_wines": "protein_concentration"}, inplace=True)